# CMAP-Elites DEMO

## Imports

In [1]:
import json

GECCO-compatible `matplotlib` options:

In [2]:
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

Import `PCGSEPy` modules:

In [3]:
from pcgsepy.common.vecs import orientation_from_str, Vec 
from pcgsepy.config import COMMON_ATOMS, HL_ATOMS, N_ITERATIONS, REQ_TILES 
from pcgsepy.lsystem.rules import RuleMaker
from pcgsepy.lsystem.actions import AtomAction, Rotations
from pcgsepy.lsystem.parser import HLParser, LLParser
from pcgsepy.lsystem.solver import LSolver
from pcgsepy.lsystem.constraints import ConstraintHandler, ConstraintLevel, ConstraintTime
from pcgsepy.lsystem.constraints_funcs import components_constraint, intersection_constraint, symmetry_constraint, axis_constraint
from pcgsepy.lsystem.lsystem import LSystem
from pcgsepy.structure import block_definitions
from pcgsepy.evo.genops import expander

## Setup

In [4]:
with open(COMMON_ATOMS, "r") as f:
    common_alphabet = json.load(f)

for k in common_alphabet:
    action, args = common_alphabet[k]["action"], common_alphabet[k]["args"]
    action = AtomAction(action)
    if action == AtomAction.MOVE:
        args = orientation_from_str[args]
    elif action == AtomAction.ROTATE:
        args = Rotations(args)
    common_alphabet[k] = {"action": action, "args": args}

In [5]:
with open(HL_ATOMS, "r") as f:
    hl_atoms = json.load(f)

tiles_dimensions = {}
tiles_block_offset = {}
for tile in hl_atoms.keys():
    dx, dy, dz = hl_atoms[tile]["dimensions"]
    tiles_dimensions[tile] = Vec.v3i(dx, dy, dz)
    tiles_block_offset[tile] = hl_atoms[tile]["offset"]

hl_alphabet = {}
for k in common_alphabet.keys():
    hl_alphabet[k] = common_alphabet[k]

for hk in hl_atoms.keys():
    hl_alphabet[hk] = {"action": AtomAction.PLACE, "args": []}

In [6]:
ll_alphabet = {}

for k in common_alphabet.keys():
    ll_alphabet[k] = common_alphabet[k]

# for k in block_definitions.keys():
#     if k != "":  # TODO: This is a probable bug, reported to the SE API devs
#         ll_alphabet[k] = {"action": AtomAction.PLACE, "args": [k]}

In [7]:
used_ll_blocks = [
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerInv',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
    'MyObjectBuilder_Gyro_LargeBlockGyro',
    'MyObjectBuilder_Reactor_LargeBlockSmallGenerator',
    'MyObjectBuilder_CargoContainer_LargeBlockSmallContainer',
    'MyObjectBuilder_Cockpit_OpenCockpitLarge',
    'MyObjectBuilder_Thrust_LargeBlockSmallThrust',
    'MyObjectBuilder_InteriorLight_SmallLight',
    'MyObjectBuilder_CubeBlock_Window1x1Slope',
    'MyObjectBuilder_CubeBlock_Window1x1Flat',
    'MyObjectBuilder_InteriorLight_LargeBlockLight_1corner'
]

for k in used_ll_blocks:
    ll_alphabet[k] = {"action": AtomAction.PLACE, "args": [k]}

## L-System components

In [8]:
hl_rules = RuleMaker(ruleset='hlrules').get_rules()
ll_rules = RuleMaker(ruleset='llrules').get_rules()

hl_parser = HLParser(rules=hl_rules)
ll_parser = LLParser(rules=ll_rules)

hl_solver = LSolver(parser=hl_parser,
                    atoms_alphabet=hl_alphabet,
                    extra_args={
                        'tiles_dimensions': tiles_dimensions,
                        'tiles_block_offset': tiles_block_offset,
                        'll_rules': ll_rules
                    })
ll_solver = LSolver(parser=ll_parser,
                    atoms_alphabet=dict(hl_alphabet, **ll_alphabet),
                    extra_args={})

In [9]:
rcc1 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc1.extra_args["req_tiles"] = ['cockpit']

rcc2 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc2.extra_args["req_tiles"] = ['corridorcargo', 'corridorgyros', 'corridorreactors']

rcc3 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc3.extra_args["req_tiles"] = ['thrusters']

nic = ConstraintHandler(
    name="no_intersections",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.DURING,
    f=intersection_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    },
    needs_ll=True
)
nic.extra_args["tiles_dimensions"] = tiles_dimensions

sc = ConstraintHandler(
    name="symmetry",
    level=ConstraintLevel.SOFT_CONSTRAINT,
    when=ConstraintTime.END,
    f=symmetry_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    }
)

In [10]:
lsystem = LSystem(
    hl_solver=hl_solver, ll_solver=ll_solver, names=['HeadModule', 'BodyModule', 'TailModule']
)

In [11]:
lsystem.add_hl_constraints(cs=[
    [nic, rcc1],
    [nic, rcc2],
    [nic, rcc3]
])

lsystem.add_ll_constraints(cs=[
    [sc],
    [sc],
    [sc]
])

In [12]:
expander.initialize(rules=lsystem.hl_solver.parser.rules)

## MAP-Elites

In [13]:
from pcgsepy.evo.fitness import box_filling_fitness, bounding_box_fitness, func_blocks_fitness, axis_fitness

feasible_fitnesses = [bounding_box_fitness,
                      box_filling_fitness,
                      func_blocks_fitness,
                      axis_fitness]

In [14]:
from pcgsepy.mapelites.map import MAPElites

In [15]:
mapelites = MAPElites(lsystem=lsystem,
                      feasible_fitnesses=feasible_fitnesses,
                      behavior_limits=(20, 20),
                      n_bins=(8, 8))

In [16]:
# mapelites.generate_initial_populations(pops_size=20,
#                                        n_retries=100)

# mapelites.show_fitness(show_mean=True,
#                        population='feasible')
# mapelites.show_fitness(show_mean=False,
#                        population='feasible')
# mapelites.show_fitness(show_mean=True,
#                        population='infeasible')
# mapelites.show_fitness(show_mean=False,
#                        population='infeasible')
# mapelites.show_coverage(population='feasible')
# mapelites.show_coverage(population='infeasible')
# mapelites.show_age(show_mean=True,
#                    population='feasible')
# mapelites.show_age(show_mean=True,
#                    population='infeasible')
# mapelites.show_age(show_mean=False,
#                    population='feasible')
# mapelites.show_age(show_mean=False,
#                    population='infeasible')

In [17]:
# from tqdm.notebook import trange

# for i in trange(2):
#     mapelites.rand_step(gen=i)

# # mapelites.interactive_mode(n_steps=5)

# mapelites.show_fitness(show_mean=True,
#                        population='feasible')
# mapelites.show_fitness(show_mean=False,
#                        population='feasible')
# mapelites.show_fitness(show_mean=True,
#                        population='infeasible')
# mapelites.show_fitness(show_mean=False,
#                        population='infeasible')
# mapelites.show_coverage(population='feasible')
# mapelites.show_coverage(population='infeasible')
# mapelites.show_age(show_mean=True,
#                    population='feasible')
# mapelites.show_age(show_mean=True,
#                    population='infeasible')
# mapelites.show_age(show_mean=False,
#                    population='feasible')
# mapelites.show_age(show_mean=False,
#                    population='infeasible')

## DASH

In [18]:
!pip install dash dash-html-components dash-core-components

In [19]:
!pip install pandas

In [20]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import numpy as np
import plotly.express as px

import matplotlib.pyplot as plt

from typing import Tuple

In [65]:
def from_bc_to_idx(bcs: Tuple[float, float], me: MAPElites) -> Tuple[int, int]:
    b0, b1 = bcs
    return (int(b0 // me.bin_sizes[0]), int(b1 // me.bin_sizes[1]))


gen_counter = 0

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__,
                title='SE ICMAP-Elites',
                external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1(children="Space Engineers ICMAP-Elites",
            style={'textAlign': 'center'}),
    html.Div([
        html.H6('Choose population:'),
        html.P('Choose which population to display.'),
        dcc.Dropdown(
            ['Feasible', 'Infeasible'],
            'Feasible',
            id='population-dropdown',
        ),
        html.H6('Choose metric:'),
        html.P('Choose which metric to plot.'),
        dcc.Dropdown(
            ['Fitness', 'Age', 'Coverage'],
            'Fitness',
            id='metric-dropdown',
        ),
        html.H6('Choose population or elitist:'),
        html.P('Choose whether to compute the metric for the entire bin population or just the elitist.'),
        dcc.RadioItems(
            ['Population', 'Elitist'],
            'Population',
            id='method-radio',
            labelStyle={'display': 'inline-block', 'marginTop': '5px'}
        )],
        style={'width': '15%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    html.Div([
        dcc.Graph(id="heatmap-plot")
    ],
        style={'width': '25%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    html.Div([
        html.H6('Experiment settings:'),
        html.P('Valid bins are: ', id='valid-bins'),
        html.P(f'Current generation: {gen_counter}', id='gen-display'),
        html.P('', id='hidden-p', style={'display': 'none'}),
        html.Button('Apply step', id='step-btn', n_clicks=0),
        html.Button('Initialize/Reset', id='reset-btn', n_clicks=0),
        html.P('', id='selected-bin')
    ],
        style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top', 'horizontalAlign': 'right'})
])

In [66]:
from pcgsepy.config import BIN_POP_SIZE, CS_MAX_AGE

hm_callback_props = {
    'pop': {
        'Feasible': 'feasible',
        'Infeasible': 'infeasible'
    },
    'metric': {
        'Fitness': {
            'name': 'fitness',
            'zmax': 4.5,  # TODO: get from mapelites
            'colorscale': 'Inferno'
        },
        'Age':  {
            'name': 'age',
            'zmax': CS_MAX_AGE,
            'colorscale': 'Greys'
        },
        'Coverage': {
            'name': 'size',
            'zmax': BIN_POP_SIZE,
            'colorscale': 'Hot'
        }
    },
    'method': {
        'Population': True,
        'Elitist': False
    }
}

In [67]:
@app.callback(Output('hidden-p', 'children'),
              Output('selected-bin', 'children'),
              Input('heatmap-plot', 'clickData'))
def display_click_data(clickData):
    if clickData is not None:
        i, j = from_bc_to_idx(bcs=(clickData['points'][0]['x'],
                                   clickData['points'][0]['y']),
                              me=mapelites)
        return f'{j},{i}', f'Selected bin is ({i}, {j})'
    else:
        return '', ''


def build_heatmap(pop_name,
                  metric_name,
                  method_name):
    metric = hm_callback_props['metric'][metric_name]
    use_mean = hm_callback_props['method'][method_name]
    population = hm_callback_props['pop'][pop_name]
    # build hotmap
    disp_map = np.zeros(shape=mapelites.bins.shape)
    for i in range(mapelites.bins.shape[0]):
        for j in range(mapelites.bins.shape[1]):
            disp_map[i, j] = mapelites.bins[i, j].get_metric(metric=metric['name'],
                                                             use_mean=use_mean,
                                                             population=population)
    # plot
    x_labels = np.arange(0, mapelites.limits[0], mapelites.bin_sizes[0])
    y_labels = np.arange(0, mapelites.limits[1], mapelites.bin_sizes[1])
    title = f'{pop_name} population {metric_name.lower()} ({"Average" if use_mean else "Elitist"})'
    heatmap = px.imshow(disp_map,
                        zmin=0,
                        zmax=hm_callback_props['metric'][metric_name]['zmax'],
                        origin='lower',
                        labels=dict(x="Largest / Smallest",
                                    y="Largest / Medium",
                                    color=metric_name),
                        x=x_labels,
                        y=y_labels,
                        title=title,
                        aspect='equal',
                        width=500,
                        height=500,
                        color_continuous_scale=hm_callback_props['metric'][metric_name]['colorscale'])
    heatmap.update_layout(clickmode='event+select')
    heatmap.update_layout(
        xaxis={
            'tickmode': 'linear',
            'tick0': 0,
            'dtick': mapelites.bin_sizes[0]
        },
        yaxis={
            'tickmode': 'linear',
            'tick0': 0,
            'dtick': mapelites.bin_sizes[1]
        }
    )
    return heatmap


def get_valid_bins():
    valid_bins = [x.bin_idx for x in mapelites._valid_bins()]
    res = []
    for i, j in valid_bins:
        res.append((j, i))
    return res


@app.callback(Output('heatmap-plot', 'figure'),
              Output('valid-bins', 'children'),
              Output('gen-display', 'children'),
              State('hidden-p', 'children'),
              State('heatmap-plot', 'figure'),
              Input('population-dropdown', 'value'),
              Input('metric-dropdown', 'value'),
              Input('method-radio', 'value'),
              Input('step-btn', 'n_clicks'),
              Input('reset-btn', 'n_clicks'),)
def update_heatmap(bin_idx,
                   curr_heatmap,
                   pop_name,
                   metric_name,
                   method_name,
                   n_clicks_step,
                   n_clicks_reset):
    global gen_counter

    ctx = dash.callback_context

    if not ctx.triggered:
        event_trig = None
    else:
        event_trig = ctx.triggered[0]['prop_id'].split('.')[0]

    if event_trig == 'step-btn':
        if bin_idx is not None and bin_idx != '':
            bin_x, bin_y = bin_idx.split(',')
            bin_idx = (int(bin_x), int(bin_y))
            if bin_idx in [x.bin_idx for x in mapelites._valid_bins()]:
                mapelites._interactive_step(bin_idx=bin_idx,
                                            n_gen=gen_counter)
                gen_counter += 1
                curr_heatmap = build_heatmap(pop_name=pop_name,
                                             metric_name=metric_name,
                                             method_name=method_name)
                msg = ''
            else:
                msg = f'; bin at ({bin_x}, {bin_y}) is not valid'
            return curr_heatmap, f'Valid bins are: {get_valid_bins()}', f'Current generation: {gen_counter}' + msg
        else:
            return curr_heatmap, f'Valid bins are: {get_valid_bins()}', f'Current generation: {gen_counter}'
    elif event_trig == 'reset-btn':
        gen_counter = 0
        mapelites.reset()
        heatmap = build_heatmap(pop_name=pop_name,
                                metric_name=metric_name,
                                method_name=method_name)
        return heatmap, f'Valid bins are: {get_valid_bins()}', f'Current generation: {gen_counter}'
    else:
        heatmap = build_heatmap(pop_name=pop_name,
                                metric_name=metric_name,
                                method_name=method_name)
        return heatmap, f'Valid bins are: {get_valid_bins()}', f'Current generation: {gen_counter}'

In [68]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
